# **Entrenamiento de modelo con tecnica Word2Vec**

In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00


**Corpus**: Documento extenso que contiene instrucciones del lenguaje

In [6]:
from gensim.models import Word2Vec # gensim modelo que se va a entrenar y que entienda de embedings a diferentes palabras para ello se usara Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



# Cargar los datos que vamos a utilizar desde un data set de huggingface llamado large_spanish_corpus especificamente se usara el corpus ParaCrawl

In [7]:
from datasets import load_dataset
dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for large_spanish_corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/large_spanish_corpus
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.


Generating train split:   0%|          | 0/15510649 [00:00<?, ? examples/s]

# ver los datos de entrenamientos descargados y separados en el objeto dataset_corpus

Trabajar con corpus con mas completos el anterior es muy pequeno

In [8]:
dataset_corpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15510649
    })
})

In [9]:
subset = dataset_corpus['train'].select(range(1000000))

In [10]:
subset[0:2]

{'text': ['lavado de cerebro a través de los medios de comunicación, y amenaza de fuerza a través de los militares.',
  'Sin un constante aluvión de doble cañón, requiriendo la complicidad de los seres humanos para reprimir y engañar a sus semejantes, su tan cacareada magia rápidamente se desvanecería y se disiparía.']}

# Librerias para procesar y limpiar la informacion del corpus

In [11]:
import nltk
nltk.download('stopwords') # stopwords: son las palabras que no aportan valor al dataset
nltk.download('punkt') # punkt: sirve para tokenizar las oraciones
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Preprocesamiento del corpus

Cada corpus se limpia de distinta manera

In [12]:
# STOP_WORDS = set(stopwords.words('spanish')) # 1


def clean_text(sentence_batch):
  # extrae el texto de la entrada
  text_list = sentence_batch['text']
  cleaned_text_list = []
  for text in text_list:

    # Alternative to next code
    # filtered_text = [word for word in word_tokens if word not in STOP_WORDS] # 2


    # convierte el texto a minusculas
    text = text.lower()

    # Elimina URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Elimina las menciones @ y '#' de las redes sociales
    text = re.sub(r'\@\W+|\#\w+', '', text)

    # Elimina los caracteres de puntuacion
    text = strip_punctuation(text)

    # Elimina los numeros
    text = strip_numeric(text)

    # Elimina las palabras cortas
    text = strip_short(text, minsize=2)

    # Elimina las palabras comunes (stop words)
    stop_words = set(stopwords.words('spanish'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]

    cleaned_text_list.append(filtered_text)

  # Devolver el texto limpio
  return {'text': cleaned_text_list}

# Funcion de procesamiento de lenguaje al corpus

In [13]:
sentences_corpus = subset.map(clean_text, batched=True)

Parameter 'function'=<function clean_text at 0x7848b8c615a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Resultado

In [14]:
sentences_corpus['text'][:3]

[['lavado',
  'cerebro',
  'través',
  'medios',
  'comunicación',
  'amenaza',
  'fuerza',
  'través',
  'militares'],
 ['constante',
  'aluvión',
  'doble',
  'cañón',
  'requiriendo',
  'complicidad',
  'seres',
  'humanos',
  'reprimir',
  'engañar',
  'semejantes',
  'tan',
  'cacareada',
  'magia',
  'rápidamente',
  'desvanecería',
  'disiparía'],
 ['realidad',
  'nuevo',
  'om',
  'sólo',
  'puede',
  'mantener',
  'ilusión',
  'supremacía',
  'mágica',
  'siempre',
  'reprima',
  'desvíe',
  'potencial',
  'humano',
  'mora',
  'verdadera',
  'magia',
  'decir',
  'capacidad',
  'innata',
  'especie',
  'magia',
  'interactiva',
  'poderes',
  'animación',
  'diosa',
  'planetaria']]

# Cargar y uso de modelo de embeddings Word2Vec para pasar palabras a vectores

## Generando el entrenamiento

In [15]:
model = Word2Vec(sentences_corpus['text'], vector_size=100, window=5, min_count=2, workers=6, sg=1) # sentences_corpus['text']:  Lo entrenamos con sentences_corpus, vector_size: definir la dimencion del tamano que vamos a tener

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

Prueba de la vectorizacion

In [16]:
model.wv['rey']

array([-0.4610543 , -0.27103743,  0.41612458, -0.0772217 , -0.3222829 ,
       -0.6411781 , -0.09712014,  0.63252586, -0.20992462, -0.03492338,
       -0.58232695, -0.51404065, -0.1826761 ,  0.45309436, -0.64067173,
        0.37840816, -0.6326861 , -0.5955528 ,  0.07974403, -0.20704359,
        0.73734707,  0.5677743 ,  0.56223613, -0.6146745 ,  0.2652858 ,
        0.4740251 , -0.25705096, -0.45315808, -0.76504225,  0.8116412 ,
       -0.19440795,  0.63228786,  0.09159822, -0.6529887 , -0.14195596,
        0.5759406 , -0.4485757 , -0.40488493, -0.08585797, -0.59601855,
        0.3359568 ,  0.3109195 ,  0.44628015, -0.39745113, -0.16038   ,
       -0.09169398, -0.4924173 , -0.5080627 , -0.53304744,  0.60702455,
        0.11088789, -0.31479147,  0.08445802, -0.23528515, -0.36583024,
       -0.1382178 , -0.07667057, -0.24218595,  0.34057248,  0.20524912,
        0.00551634, -0.19475912,  0.15512054, -0.17066355,  0.46337822,
        0.6362341 , -0.0767953 ,  0.0233404 , -0.6244598 ,  0.02

### Obtener palabras similares de la vectorizacion anterior

In [22]:
model.wv.most_similar(['comida'], topn=5)

# model.wv.most_similar(['ser'], topn=5)

# model.wv.most_similar(['reina'], topn=5)

# model.wv.most_similar(['television'], topn=5)

[('sofía', 0.7434273958206177),
 ('semíramis', 0.7394866943359375),
 ('thyssen', 0.7325616478919983),
 ('peinador', 0.7245931625366211),
 ('semiramis', 0.7169655561447144)]

### Separar wv, vectores y palabras

In [25]:
word_vectors = model.wv
vectors = word_vectors.vectors
words = word_vectors.index_to_key

In [24]:
vectors

array([[-0.09881608,  0.13606925,  0.34970576, ..., -0.6213868 ,
        -0.10160705,  0.43461993],
       [-0.10788522,  0.2783179 ,  0.40495312, ..., -0.6188768 ,
        -0.5262344 , -0.17188407],
       [-0.21885726, -0.36157325, -0.11840174, ..., -0.5667853 ,
        -0.36615565, -0.40670744],
       ...,
       [ 0.02664475,  0.04440436,  0.10919726, ..., -0.21813567,
        -0.05621288, -0.01469944],
       [ 0.06313498,  0.1248974 ,  0.06954513, ..., -0.1629998 ,
        -0.04477697,  0.03759217],
       [ 0.11182125,  0.13640857,  0.10367149, ..., -0.1974668 ,
         0.01869208,  0.03871165]], dtype=float32)

# Almacenamiento de embeddings

In [26]:
df_vectors = pd.DataFrame(vectors)
df_vectors.to_csv('embeddings.tsv', sep='\t', index=False) # se guarda el csv en embeddings.tsv son separador de espacio \t en lugar de coma y no tendra el indece del vector vectorizado

In [29]:
df_words = pd.DataFrame(words)
df_words.to_csv('labels.tsv', sep='\t', index=False)

# Usar https://projector.tensorflow.org/ para convertir en 2 o 3 dimenciones los vectores